## reading in data and importing things

In [1]:
#importing things
import pandas as pd
import os
import numpy as np
import csv
import pylab as pl
import matplotlib.pyplot as plt
import copy
import scipy
import scipy.stats as stats
from scipy.stats import linregress
from scipy.stats import t
from scipy.stats import chi2
from scipy.stats import sem
from scipy.linalg import hadamard
from scipy.stats import ttest_ind_from_stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import ttest_ind_from_stats
from sklearn.manifold import TSNE
from collections import OrderedDict
import seaborn as sns
from functools import reduce
from scipy.optimize import curve_fit
from operator import itemgetter
import math

In [2]:
#reading in counts
rootdir = 'expression_reads/9114g'
list_of_dataframes = []
for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if '9114' in file:
                filepath = str(rootdir)+'/'+str(file)
                filename, file_extension = os.path.splitext(file)
                print(filepath)
                list_of_dataframes.append(pd.read_table(filepath,delimiter='\t\t',names=['variant',filename],index_col=None,dtype={'variant': str,'LogEC50':float}))

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['variant'],
                                            how='outer'), list_of_dataframes)
df_merged.fillna(0, inplace=True)
pd.DataFrame.to_csv(df_merged, 'expression_analysis/9114g_merged_reads.txt', sep=',', na_rep=float(0), index=False)
df_merged

expression_reads/9114g/counts_9114_g_F_8.txt


/Users/angelaphillips/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


expression_reads/9114g/counts_9114_g_F_1.txt
expression_reads/9114g/counts_9114_g_F_2.txt
expression_reads/9114g/counts_9114_g_F_3.txt
expression_reads/9114g/counts_9114_g_F_7.txt
expression_reads/9114g/counts_9114_g_F_6.txt
expression_reads/9114g/counts_9114_g_F_4.txt
expression_reads/9114g/counts_9114_g_F_5.txt


,variant,counts_9114_g_F_8,counts_9114_g_F_1,counts_9114_g_F_2,counts_9114_g_F_3,counts_9114_g_F_7,counts_9114_g_F_6,counts_9114_g_F_4,counts_9114_g_F_5
0,1101010000111101,1755.0,136.0,157.0,598.0,1000.0,569.0,1468.0,976.0
1,1111101010000111,1213.0,86.0,138.0,594.0,949.0,546.0,707.0,849.0
2,1111110010011111,1210.0,93.0,63.0,317.0,371.0,184.0,358.0,313.0
3,1110010000110111,1130.0,131.0,76.0,369.0,557.0,436.0,970.0,804.0
4,1100010110110001,1093.0,308.0,80.0,767.0,913.0,495.0,1120.0,987.0
...,...,...,...,...,...,...,...,...,...
65531,0010100011110110,0.0,0.0,0.0,0.0,1.0,1.0,0.0,6.0
65532,0010100110001000,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0
65533,0000100111110101,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0
65534,0010100010110100,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0


In [3]:
#extracting mean log fluor and stdev log fluor from fcs files
rootdir = '20200821_gate_csv/9114g'

concs = ["myc_FITC"]
FITC_min_list=[]
FITC1_bin_avg_list=[]
FITC2_bin_avg_list=[]
FITC3_bin_avg_list=[]
FITC4_bin_avg_list=[]
FITC5_bin_avg_list=[]
FITC6_bin_avg_list=[]
FITC7_bin_avg_list=[]
FITC8_bin_avg_list=[]
FITC1_bin_std_list=[]
FITC2_bin_std_list=[]
FITC3_bin_std_list=[]
FITC4_bin_std_list=[]
FITC5_bin_std_list=[]
FITC6_bin_std_list=[]
FITC7_bin_std_list=[]
FITC8_bin_std_list=[]
for conc in concs:
    #calc min PE fluor to shift all above zero
    FITC_min = 0
    FITC_list=[]
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            filepath = subdir + os.sep + file
            if conc in filepath:
                print(filepath)
                with open(filepath, encoding="utf8", errors="ignore") as csv_file:
                    csv_reader = csv_file.readlines()
                    for line in csv_reader:
                        FITC_list.append(float(line.split(",")[6]))

    if min(FITC_list) < FITC_min:
        FITC_min = min(FITC_list)
    else:
        FITC_min = FITC_min
    print("FITC_min: "+str(conc)+":"+str(FITC_min))
    FITC_min_list.append(FITC_min)
    
    #bin mean log fluor
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + file
            if conc in filepath:
                if "FITC1" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC1_bin_avg: "+str(bin_avg))
                        print(conc+"FITC1_bin_std: "+str(bin_std))
                        FITC1_bin_avg_list.append(bin_avg)
                        FITC1_bin_std_list.append(bin_std)
                elif "FITC2" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC2_bin_avg: "+str(bin_avg))
                        print(conc+"FITC2_bin_std: "+str(bin_std))
                        FITC2_bin_avg_list.append(bin_avg)
                        FITC2_bin_std_list.append(bin_std)
                elif "FITC3" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC3_bin_avg: "+str(bin_avg))
                        print(conc+"FITC3_bin_std: "+str(bin_std))
                        FITC3_bin_avg_list.append(bin_avg)
                        FITC3_bin_std_list.append(bin_std)
                elif "FITC4" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC4_bin_avg: "+str(bin_avg))
                        print(conc+"FITC4_bin_std: "+str(bin_std))
                        FITC4_bin_avg_list.append(bin_avg)
                        FITC4_bin_std_list.append(bin_std)
                elif "FITC5" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC5_bin_avg: "+str(bin_avg))
                        print(conc+"FITC5_bin_std: "+str(bin_std))
                        FITC5_bin_avg_list.append(bin_avg)
                        FITC5_bin_std_list.append(bin_std)
                elif "FITC6" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC6_bin_avg: "+str(bin_avg))
                        print(conc+"FITC6_bin_std: "+str(bin_std))
                        FITC6_bin_avg_list.append(bin_avg)
                        FITC6_bin_std_list.append(bin_std)
                elif "FITC7" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC7_bin_avg: "+str(bin_avg))
                        print(conc+"FITC7_bin_std: "+str(bin_std))
                        FITC7_bin_avg_list.append(bin_avg)
                        FITC7_bin_std_list.append(bin_std)
                elif "FITC8" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC8_bin_avg: "+str(bin_avg))
                        print(conc+"FITC8_bin_std: "+str(bin_std))
                        FITC8_bin_avg_list.append(bin_avg)
                        FITC8_bin_std_list.append(bin_std)
    
with open('expression_analysis/fluor_mean_std_9114g.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerows(zip(concs,FITC_min_list,FITC1_bin_avg_list,FITC2_bin_avg_list,FITC3_bin_avg_list,FITC4_bin_avg_list,FITC5_bin_avg_list,FITC6_bin_avg_list,FITC7_bin_avg_list,FITC8_bin_avg_list,FITC1_bin_std_list,FITC2_bin_std_list,FITC3_bin_std_list,FITC4_bin_std_list,FITC5_bin_std_list,FITC6_bin_std_list,FITC7_bin_std_list,FITC8_bin_std_list))

20200821_gate_csv/9114g/export_Sorted_9114g_myc_FITC_FITC7.csv
20200821_gate_csv/9114g/export_Sorted_9114g_myc_FITC_FITC6.csv
20200821_gate_csv/9114g/export_Sorted_9114g_myc_FITC_FITC4.csv
20200821_gate_csv/9114g/export_Sorted_9114g_myc_FITC_FITC5.csv
20200821_gate_csv/9114g/export_Sorted_9114g_myc_FITC_FITC1.csv
20200821_gate_csv/9114g/export_Sorted_9114g_myc_FITC_FITC2.csv
20200821_gate_csv/9114g/export_Sorted_9114g_myc_FITC_FITC3.csv
20200821_gate_csv/9114g/export_Sorted_9114g_myc_FITC_FITC8.csv
FITC_min: myc_FITC:-163.059
20200821_gate_csv/9114g/export_Sorted_9114g_myc_FITC_FITC7.csv
myc_FITCFITC7_bin_avg: 4.111306340678464
myc_FITCFITC7_bin_std: 0.048697562183318374
20200821_gate_csv/9114g/export_Sorted_9114g_myc_FITC_FITC6.csv
myc_FITCFITC6_bin_avg: 3.9521024694634126
myc_FITCFITC6_bin_std: 0.04495439270470931
20200821_gate_csv/9114g/export_Sorted_9114g_myc_FITC_FITC4.csv
myc_FITCFITC4_bin_avg: 3.5645350578375194
myc_FITCFITC4_bin_std: 0.08714913316701611
20200821_gate_csv/9114g/

In [4]:
#reading in fluor values and cell counts
fluor = pd.DataFrame(pd.read_csv('expression_analysis/fluor_mean_std_9114g.csv',delimiter=',',names=['concen','FITCmin','1','2','3','4','5','6','7','8','1s','2s','3s','4s','5s','6s','7s','8s']))
cellct = pd.DataFrame(pd.read_csv('cell_counts/9114g_cellct_exp.csv',delimiter=','))

## calculate mean bin and read counts

In [5]:
concs = ['_F_']
i = 0 #index for conc
total_ct = []
bin1_cts = []
bin2_cts = []
bin3_cts = []
bin4_cts = []
bin5_cts = []
bin6_cts = []
bin7_cts = []
bin8_cts = []
for conc in concs:
    #first grab the counts and the fluor values
    #total cell counts in bin (a,b,c,d)
    Ca = float(cellct.at[i,'1'])
    Cb = float(cellct.at[i,'2'])
    Cc = float(cellct.at[i,'3'])
    Cd = float(cellct.at[i,'4'])
    Ce = float(cellct.at[i,'5'])
    Cf = float(cellct.at[i,'6'])
    Cg = float(cellct.at[i,'7'])
    Ch = float(cellct.at[i,'8'])
    bin1_ct = df_merged['counts_9114_g'+str(conc)+'1'].sum(axis=0) #sum all counts in a bin
    avg_fluor_1 = fluor.at[i, '1'] #extract mean fluor
    bin2_ct = df_merged['counts_9114_g'+str(conc)+'2'].sum(axis=0) #sum all counts in a bin
    avg_fluor_2 = fluor.at[i, '2'] #extract mean fluor
    bin3_ct = df_merged['counts_9114_g'+str(conc)+'3'].sum(axis=0) #sum all counts in a bin
    avg_fluor_3 = fluor.at[i, '3'] #extract mean fluor
    bin4_ct = df_merged['counts_9114_g'+str(conc)+'4'].sum(axis=0) #sum all counts in a bin
    avg_fluor_4 = fluor.at[i, '4'] #extract mean fluor
    bin5_ct = df_merged['counts_9114_g'+str(conc)+'5'].sum(axis=0) #sum all counts in a bin
    avg_fluor_5 = fluor.at[i, '5'] #extract mean fluor
    bin6_ct = df_merged['counts_9114_g'+str(conc)+'6'].sum(axis=0) #sum all counts in a bin
    avg_fluor_6 = fluor.at[i, '6'] #extract mean fluor
    bin7_ct = df_merged['counts_9114_g'+str(conc)+'7'].sum(axis=0) #sum all counts in a bin
    avg_fluor_7 = fluor.at[i, '7'] #extract mean fluor
    bin8_ct = df_merged['counts_9114_g'+str(conc)+'8'].sum(axis=0) #sum all counts in a bin
    avg_fluor_8 = fluor.at[i, '8'] #extract mean fluor
    total_ct_c = bin1_ct + bin2_ct + bin3_ct + bin4_ct + bin5_ct + bin6_ct + bin7_ct + bin8_ct #sum total count
    total_ct.append(total_ct_c) #append to list
    bin1_cts.append(bin1_ct)
    bin2_cts.append(bin2_ct)
    bin3_cts.append(bin3_ct)
    bin4_cts.append(bin4_ct)
    bin5_cts.append(bin5_ct)
    bin6_cts.append(bin6_ct)
    bin7_cts.append(bin7_ct)
    bin8_cts.append(bin8_ct)
    i = i + 1
    #now calc the var frequency
    df_merged['mean_bin1_%s'%conc] = (100*Ca*df_merged['counts_9114_g'+str(conc)+'1']/bin1_ct)
    df_merged['mean_bin2_%s'%conc] = (100*Cb*df_merged['counts_9114_g'+str(conc)+'2']/bin2_ct)
    df_merged['mean_bin3_%s'%conc] = (100*Cc*df_merged['counts_9114_g'+str(conc)+'3']/bin3_ct)
    df_merged['mean_bin4_%s'%conc] = (100*Cd*df_merged['counts_9114_g'+str(conc)+'4']/bin4_ct)
    df_merged['mean_bin5_%s'%conc] = (100*Ce*df_merged['counts_9114_g'+str(conc)+'5']/bin5_ct)
    df_merged['mean_bin6_%s'%conc] = (100*Cf*df_merged['counts_9114_g'+str(conc)+'6']/bin6_ct)
    df_merged['mean_bin7_%s'%conc] = (100*Cg*df_merged['counts_9114_g'+str(conc)+'7']/bin7_ct)
    df_merged['mean_bin8_%s'%conc] = (100*Ch*df_merged['counts_9114_g'+str(conc)+'8']/bin8_ct)
    #now calc the meanbin
    df_merged['meanbin_%s'%conc] = (df_merged['mean_bin1_%s'%conc] * float(avg_fluor_1) + df_merged['mean_bin2_%s'%conc] * float(avg_fluor_2) + df_merged['mean_bin3_%s'%conc] * float(avg_fluor_3) + df_merged['mean_bin4_%s'%conc] * float(avg_fluor_4) + df_merged['mean_bin5_%s'%conc] * float(avg_fluor_5) + df_merged['mean_bin6_%s'%conc] * float(avg_fluor_6) + df_merged['mean_bin7_%s'%conc] * float(avg_fluor_7) + df_merged['mean_bin8_%s'%conc] * float(avg_fluor_8))/(df_merged['mean_bin1_%s'%conc]+df_merged['mean_bin2_%s'%conc]+df_merged['mean_bin3_%s'%conc]+df_merged['mean_bin4_%s'%conc]+df_merged['mean_bin5_%s'%conc]+df_merged['mean_bin6_%s'%conc]+df_merged['mean_bin7_%s'%conc]+df_merged['mean_bin8_%s'%conc])
df_totals = pd.DataFrame(list(zip(concs, total_ct, bin1_cts, bin2_cts, bin3_cts, bin4_cts, bin5_cts, bin6_cts, bin7_cts, bin8_cts)), columns =['concen', 'total_ct', '1','2','3','4','5', '6','7','8'])
df_merged
pd.DataFrame.to_csv(df_merged,'expression_analysis/expression_meanbins_9114g.csv')